In [1]:
pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import os

In [5]:
# File paths
import os
import pandas as pd

RAW_FILE = r"C:\Users\Hp\Desktop\service-fleet-analytics\data_raw\cincinnati_fleet_work_orders.csv.csv"
OUT_DIR = r"C:\Users\Hp\Desktop\service-fleet-analytics\data_clean"

os.makedirs(OUT_DIR, exist_ok=True)



In [6]:
#Load data
df = pd.read_csv(RAW_FILE, low_memory=False)
print(df.shape)

(332420, 37)


In [7]:
df.columns

Index(['UNIQUE_WORK_ORDER_NO', 'CREATE_DATE', 'LOC_WORK_ORDER_LOC',
       'LOC_WORK_ORDER_LOC_NAME', 'WORK_ORDER_YR', 'WORK_ORDER_NO', 'JOB_TYPE',
       'EQ_EQUIP_NO', 'WORK_ORDER_STATUS', 'METER_1_READING',
       'DATETIME_OUT_SERVICE', 'DATETIME_IN_SERVICE', 'DATETIME_OPEN',
       'DATETIME_FIRST_LABOR', 'DATETIME_FINISHED', 'DATETIME_CLOSED',
       'DATETIME_UNIT_IN', 'DATETIME_DUE', 'DATETIME_PM_SCHED',
       'QTY_EST_HOURS', 'DOWNTIME_HRS_USER', 'DOWNTIME_HRS_SHOP', 'WARRANTY',
       'REAS_REAS_FOR_REPAIR', 'REAS_FOR_REPAIR_DESC', 'PRI_PRIORITY_CODE',
       'REF_WORK_ORDER_NO', 'DEPT_EQUIP_DEPT', 'DEPT_EQUIP_DEPT_NAME',
       'METER_1_LIFE_TOTAL', 'EQ_PARENT_EQUIP_NO', 'DELAY_HOURS',
       'LABOR_HOURS', 'LABOR_COST', 'PARTS_COST', 'COMML_COST', 'TOTAL_COST'],
      dtype='object')

In [8]:
# Keep only needed columns 
df = df[[
    "UNIQUE_WORK_ORDER_NO",
    "EQ_EQUIP_NO",
    "JOB_TYPE",
    "WORK_ORDER_STATUS",
    "DATETIME_OPEN",
    "DATETIME_CLOSED",
    "DEPT_EQUIP_DEPT_NAME",
    "TOTAL_COST"
]].copy()

In [9]:
# Rename to simple names
df.columns = [
    "work_order_id",
    "asset_id",
    "job_type",
    "status",
    "opened_at",
    "closed_at",
    "department",
    "total_cost"
]


In [11]:
# Convert dates
df["opened_at"] = pd.to_datetime(df["opened_at"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
df["closed_at"] = pd.to_datetime(df["closed_at"], format="%Y-%m-%d %H:%M:%S", errors="coerce")

In [12]:
# Remove rows with missing important data
df = df.dropna(subset=["work_order_id", "asset_id", "opened_at"])

In [13]:
# Create month column for trends
df["service_month"] = df["opened_at"].dt.to_period("M").astype(str)

In [14]:
# Remove duplicate work orders
df = df.drop_duplicates(subset=["work_order_id"])

In [15]:
# Create assets table (unique assets)
assets = df[["asset_id", "department"]].drop_duplicates()

In [17]:
# Save cleaned files
df.to_csv(os.path.join(OUT_DIR, "service_events_clean.csv"), index=False)
assets.to_csv(os.path.join(OUT_DIR, "assets_clean.csv"), index=False)

print("✅ Files created:")
print(" - data_clean/service_events_clean.csv")
print(" - data_clean/assets_clean.csv")

df.head()

✅ Files created:
 - data_clean/service_events_clean.csv
 - data_clean/assets_clean.csv


,work_order_id,asset_id,job_type,status,opened_at,closed_at,department,total_cost,service_month
0,2023-3001-1390,60959,REPAIR,OPEN,2023-08-07,NaT,WATER WORKS,239.8,2023-08
1,2023-4145-273,21951,REPAIR,OPEN,2023-08-07,NaT,PARK DEPARTMENT,55,2023-08
2,2023-2205-312,21306,REPAIR,WORK FINISHED,2023-08-07,NaT,POLICE,110,2023-08
3,2023-4131-718,92830,REPAIR,CLOSED,2023-08-07,2023-08-08,NEIGHBORHOOD OPERATIONS,424.48,2023-08
4,2023-3001-1389,10797,REPAIR,WORK FINISHED,2023-08-07,NaT,WATER WORKS,88.47,2023-08
